In [80]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
####한국어 형태소 분석기####
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
okt = Okt()

In [82]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [83]:
df_18=pd.read_csv('./가격_및_뉴스_데이터/news18.csv',encoding='cp949')
df_19=pd.read_csv('./가격_및_뉴스_데이터/news19.csv',encoding='cp949')
df_20=pd.read_csv('./가격_및_뉴스_데이터/news20.csv',encoding='cp949')
df_21=pd.read_csv('./가격_및_뉴스_데이터/news21.csv',encoding='cp949')

In [84]:
df_18_price=pd.read_excel('./가격_및_뉴스_데이터/18년도 가격데이터.xlsx')
df_19_price=pd.read_excel('./가격_및_뉴스_데이터/19년도 가격데이터.xlsx')
df_20_price=pd.read_excel('./가격_및_뉴스_데이터/20년도 가격데이터.xlsx')
df_21_price=pd.read_excel('./가격_및_뉴스_데이터/21년도 가격데이터.xlsx')


In [85]:
#첫번째 행 삭제
df_18_price.drop(0,axis=0,inplace=True)
df_19_price.drop(0,axis=0,inplace=True)
df_20_price.drop(0,axis=0,inplace=True)
df_21_price.drop(0,axis=0,inplace=True)

In [86]:
df_price_all=pd.concat([df_18_price,df_19_price,df_20_price,df_21_price])

### 돼지고기 가격시간 datetime으로 변형

In [87]:
# df_price_all['도축일']=pd.to_datetime(df_price_all['도축일'])

In [88]:
df_price_all.tail()

,업체코드,업체명,농가코드,농가명,이력번호,도축일,도체번호,판정일,판정구분,성별,생체중량,중량,등지방,1차등급,최종등급,경락단가,경락금액,삼겹비율,전체,Unnamed: 19,Unnamed: 20,등지방.1,Unnamed: 22,Unnamed: 23,삼겹,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,목심,Unnamed: 33,Unnamed: 34,갈비,Unnamed: 36,Unnamed: 37,등갈비\n무게,등심,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,안심\n무게,앞다리살,Unnamed: 46,Unnamed: 47,뒷다리살,Unnamed: 49,Unnamed: 50,Unnamed: 51,항정살\n무게,등심덧살\n무게,갈매기살\n무게,총 금액,Unnamed: 18,Unnamed: 21,Unnamed: 24,Unnamed: 32,Unnamed: 35,Unnamed: 39,Unnamed: 44,Unnamed: 45,Unnamed: 48
50292,10466,도드람양돈협동조합,38139,오금영농조합법인(황인숙),130002500147,20211231,4239,20211231,1,거세,113.5,84,15,1,1,4189.0,NaN,NaN,65.5,20.84,NaN,27,23,NaN,11.22,4.78,26.87,2.12,12.27,55.7,38.82,NaN,4.34,70.98,NaN,3.56,61.88,NaN,0.82,5.34,68.99,31.4,NaN,NaN,0.94,7.9,NaN,15.58,16.56,NaN,NaN,NaN,0.48,0.36,0.26,351876.0,29.95,21,9.48,3.7,3.54,5.82,7.64,61.41,71.6
50293,10466,도드람양돈협동조합,38139,오금영농조합법인(황인숙),130002500147,20211231,4240,20211231,1,암,113.5,85,19,1+,1+,4269.0,NaN,NaN,68,19,NaN,28,25,NaN,11.52,4.22,24.47,1.88,10.84,58.58,35.6,NaN,4.6,74,NaN,3.56,65.63,NaN,0.84,6.28,73.56,26.43,NaN,NaN,1.04,8.16,NaN,16.84,17.66,NaN,NaN,NaN,0.48,0.38,0.26,362865.0,26.64,22,10.1,4.02,3.58,6.86,8.02,64.85,74.05
50294,10466,도드람양돈협동조합,38139,오금영농조합법인(황인숙),130002500147,20211231,4241,20211231,1,거세,113.5,80,14,2,2,4099.0,NaN,NaN,68.7,17.04,NaN,24,20,NaN,10.84,3.9,24.14,1.6,10.23,59.3,34.68,NaN,4.3,74.44,NaN,3.46,64.69,NaN,0.8,5.68,74.28,25.63,NaN,NaN,0.94,7.82,NaN,15.66,16.52,NaN,NaN,NaN,0.46,0.36,0.26,327920.0,25.77,18,9.58,3.74,3.46,6.16,7.72,64.7,74.04
50295,10466,도드람양돈협동조합,38139,오금영농조합법인(황인숙),130002500147,20211231,4242,20211231,1,거세,113.5,89,24,1+,1+,4329.0,NaN,NaN,63.9,23.48,NaN,30,27,NaN,12.66,5.12,27,2.5,13.39,54.12,40.47,NaN,4.52,69.55,NaN,3.68,62.56,NaN,0.86,5.7,66.61,33.09,NaN,NaN,0.98,8.2,NaN,16.02,17.16,NaN,NaN,NaN,0.5,0.42,0.26,385281.0,31.26,25,10.12,3.84,3.66,6.3,7.86,60.54,70.5
50296,10466,도드람양돈협동조합,38139,오금영농조합법인(황인숙),130002500147,20211231,4243,20211231,1,암,113.5,92,23,1+,1+,4399.0,NaN,NaN,62.1,25.84,NaN,33,30,NaN,13.02,5.68,28.45,2.86,14.51,52.15,42.78,NaN,4.56,67.63,NaN,3.76,60.88,NaN,0.9,5.56,63.26,36.88,NaN,NaN,0.98,8.32,NaN,15.96,17.26,NaN,NaN,NaN,0.52,0.4,0.26,404708.0,33.59,27,10.16,3.84,3.72,6.18,7.88,58.69,68.77


In [89]:
# df_price_all['datetime']=df_price_all['도축일'].dt.strftime('%Y-%m-%d')

# df_price_all['datetime']=pd.to_datetime(df_price_all['datetime'])

### 도축일 일별 기준 groupby 평균하기

In [ ]:
df_price_all_2column=df_price_all[['도축일',"경락단가"]]

df_price_all_2column['도축일']=pd.to_datetime(df_price_all_2column['도축일'])

df_price_all_2column.head()

In [ ]:
df_price_all_2column['경락단가_평균'] = df_price_all_2column.groupby(['도축일'])['경락단가'].transform('mean')

In [90]:
df_price_all_2column.head()

C:\Users\jaemin\AppData\Local\Temp\ipykernel_10156\82678589.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price_all_2column['도축일']=pd.to_datetime(df_price_all_2column['도축일'])


,도축일,경락단가
1,2018-01-02,3622.0
2,2018-01-02,3755.0
3,2018-01-02,3700.0
4,2018-01-02,3733.0
5,2018-01-02,2199.0


In [106]:
df_price_all_2column=df_price_all_2column.groupby(df_price_all_2column['도축일']).mean()

df_price_all_2column.reset_index(inplace=True)

new_df_price_all_2column=df_price_all_2column.iloc[238:,]

new_df_price_all_2column.reset_index(drop=True, inplace=True)

In [107]:
new_df_price_all_2column

,도축일,경락단가,경락단가_평균
0,2018-01-02,3750.366093,3750.366093
1,2018-01-03,3547.574932,3547.574932
2,2018-01-04,3696.060897,3696.060897
3,2018-01-05,3770.156364,3770.156364
4,2018-01-08,3809.974684,3809.974684
5,2018-01-09,3723.371930,3723.371930
6,2018-01-10,3694.435484,3694.435484
7,2018-01-11,3740.687243,3740.687243
8,2018-01-12,3748.212245,3748.212245
9,2018-01-15,3800.211009,3800.211009


### 주별로 groupby해서 평균내기

In [123]:
week = [g.reset_index() for n, g in new_df_price_all_2column.set_index('도축일').groupby(pd.Grouper(freq='W'))]

In [125]:
week[0]

,도축일,경락단가,경락단가_평균
0,2018-01-02,3750.366093,3750.366093
1,2018-01-03,3547.574932,3547.574932
2,2018-01-04,3696.060897,3696.060897
3,2018-01-05,3770.156364,3770.156364


In [129]:
new_mean=new_df_price_all_2column.set_index('도축일').groupby(pd.Grouper(freq='W')).mean()

In [132]:
new_mean=new_mean.reset_index()

In [145]:
#결측치 바로 위에 값하고 동일하게 처리
new_mean=new_mean.fillna(method='ffill')

In [146]:
new_mean['label']=0

In [148]:
new_mean

,도축일,경락단가,경락단가_평균,label
0,2018-01-07,3691.039572,3691.039572,0
1,2018-01-14,3743.336317,3743.336317,0
2,2018-01-21,3743.359986,3743.359986,0
3,2018-01-28,3835.679915,3835.679915,0
4,2018-02-04,4276.198330,4276.198330,0
5,2018-02-11,4724.707319,4724.707319,0
6,2018-02-18,4103.632481,4103.632481,0
7,2018-02-25,3794.280663,3794.280663,0
8,2018-03-04,4094.180790,4094.180790,0
9,2018-03-11,3868.237571,3868.237571,0


### 전 주와 비교하여 감소:2, 유지:0, 증가:1

In [151]:
label_list=[]
for i in range(len(new_mean)):
    if(i!=0):
        if(new_mean['경락단가'][i-1] > new_mean['경락단가'][i]):
            new_mean['label'][i]=2
        elif (new_mean['경락단가'][i-1] < new_mean['경락단가'][i]):
            new_mean['label'][i]=1
        elif new_mean['경락단가'][i-1]==new_mean['경락단가'][i]:              
            new_mean['label'][i]=0
    else:
        new_mean['label'][i]=0

C:\Users\jaemin\AppData\Local\Temp\ipykernel_10156\1809203011.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mean['label'][i]=0
C:\Users\jaemin\AppData\Local\Temp\ipykernel_10156\1809203011.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mean['label'][i]=1
C:\Users\jaemin\AppData\Local\Temp\ipykernel_10156\1809203011.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mean['label'][i]=2
C:\Users\jaemin\AppData\Loc

In [152]:
new_mean

,도축일,경락단가,경락단가_평균,label
0,2018-01-07,3691.039572,3691.039572,0
1,2018-01-14,3743.336317,3743.336317,1
2,2018-01-21,3743.359986,3743.359986,1
3,2018-01-28,3835.679915,3835.679915,1
4,2018-02-04,4276.198330,4276.198330,1
5,2018-02-11,4724.707319,4724.707319,1
6,2018-02-18,4103.632481,4103.632481,2
7,2018-02-25,3794.280663,3794.280663,2
8,2018-03-04,4094.180790,4094.180790,1
9,2018-03-11,3868.237571,3868.237571,2


In [153]:
len(new_mean)

209

In [155]:
new_mean.to_excel('./가격데이터_증감비교_라벨링.xlsx',index=False)

### 월별 키워드 분석 맵핑

In [ ]:
for i in range(1,13):
    stem_list=[]
    stem_list2=[]
    stem_list3=[]

    for text in df_18[df_18['월별']==month_list[i]]['article']:
        stem_list.append(okt.phrases(text))

    with open('./stem_list.txt','w',encoding='UTF-8') as f:
            f.write('\n'.join(map(str, stem_list)))

    f = open("./stem_list.txt", 'r',encoding='UTF-8')

    lines = f.readlines()

    for text in lines:
        malist=okt.pos(text, norm=True, stem=True)
        r=[]
        for word in malist:
            if not word[1] in ["Josa","Eomi","Punctuation", "Number",'\r','\t','Foreign','unknown']:
                r.append(word[0])
        rl=(" ".join(r)).strip()
        stem_list2.append(rl)
        print(rl)
    #     stem_list.append(okt.nouns(text))

    stem_list

    vect=CountVectorizer()

    vect.fit(stem_list2)
    globals()["new_df_{}".format(i)]=pd.DataFrame(vect.vocabulary_, index = [0])

    globals()["new_df_{}".format(i)]=globals()["new_df_{}".format(i)].T

    globals()["new_df_{}".format(i)].reset_index(inplace=True)

    globals()["new_df_{}".format(i)].columns=['word','frequency']

In [ ]:
new_df_12

In [ ]:
new_df_12_descending=new_df_12.sort_values(by=['frequency'], axis=0, ascending=False).reset_index(drop=True)

In [ ]:
new_df_12_descending

### 전월과 비교하여 공통된 단어 데이터프레임 만들기

In [ ]:
# 두 데이터프레임에서 추출한 단어들을 set 자료형으로 변환하여 비교
for i in range():
words_df1 = set(new_df_1['word'].values.flatten())
words_df2 = set(new_df_2['word'].values.flatten())
common_words = words_df1.intersection(words_df2)

common_words


In [ ]:
len(words_df2)